In [1]:
#Import Modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
import yfinance as yf
import yahoo_fin.stock_info as si
from yahoo_fin.stock_info import get_data


In [2]:
#Create data frame for S&P 500
index_name = 'SP_500'
companies = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', flavor='bs4')[0]
companies

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [3]:
#Filter data frame to pharmaceutical companies
pharma_filter = companies.loc[:,"GICS Sub-Industry"]=='Pharmaceuticals'
pharma_df = companies.loc[pharma_filter,:]
pharma_df

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
17,ALXN,Alexion Pharmaceuticals,reports,Health Care,Pharmaceuticals,"Boston, Massachusetts",2012-05-25,899866,1992
88,CTLT,Catalent,reports,Health Care,Pharmaceuticals,"Somerset, New Jersey",2020-09-21,1596783,2007
263,JNJ,Johnson & Johnson,reports,Health Care,Pharmaceuticals,"New Brunswick, New Jersey",1973-06-30,200406,1886
286,LLY,Lilly (Eli) & Co.,reports,Health Care,Pharmaceuticals,"Indianapolis, Indiana",1970-12-31,59478,1876
310,MRK,Merck & Co.,reports,Health Care,Pharmaceuticals,"Kenilworth, New Jersey",1957-03-04,310158,1891
365,PRGO,Perrigo,reports,Health Care,Pharmaceuticals,"Dublin, Ireland",2011-12-19,1585364,2013 (1887)
366,PFE,Pfizer Inc.,reports,Health Care,Pharmaceuticals,"New York, New York",1957-03-04,78003,1849
472,VTRS,Viatris,reports,Health Care,Pharmaceuticals,"Pittsburgh, Pennsylvania",2004-04-23,1792044,1961
504,ZTS,Zoetis,reports,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21,1555280,1952


In [4]:
#Add market cap
for index, stock in pharma_df.iterrows():
    symbol = pharma_df.loc[index,'Symbol']#f"{stock['Symbol']}"
    market_cap = float(si.get_quote_table(symbol)['Market Cap'].replace('B',''))
    pharma_df.loc[index,'Market Cap ($Billion)']= market_cap

#Filter by market cap
pharma_df = pharma_df.sort_values('Market Cap ($Billion)')    
pharma_df

C:\Users\Noah\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Noah\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Market Cap ($Billion)
365,PRGO,Perrigo,reports,Health Care,Pharmaceuticals,"Dublin, Ireland",2011-12-19,1585364,2013 (1887),5.827
88,CTLT,Catalent,reports,Health Care,Pharmaceuticals,"Somerset, New Jersey",2020-09-21,1596783,2007,19.680
472,VTRS,Viatris,reports,Health Care,Pharmaceuticals,"Pittsburgh, Pennsylvania",2004-04-23,1792044,1961,21.518
17,ALXN,Alexion Pharmaceuticals,reports,Health Care,Pharmaceuticals,"Boston, Massachusetts",2012-05-25,899866,1992,34.079
504,ZTS,Zoetis,reports,Health Care,Pharmaceuticals,"Florham Park, New Jersey",2013-06-21,1555280,1952,75.301
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),183.185
286,LLY,Lilly (Eli) & Co.,reports,Health Care,Pharmaceuticals,"Indianapolis, Indiana",1970-12-31,59478,1876,189.963
366,PFE,Pfizer Inc.,reports,Health Care,Pharmaceuticals,"New York, New York",1957-03-04,78003,1849,193.682
310,MRK,Merck & Co.,reports,Health Care,Pharmaceuticals,"Kenilworth, New Jersey",1957-03-04,310158,1891,199.012
263,JNJ,Johnson & Johnson,reports,Health Care,Pharmaceuticals,"New Brunswick, New Jersey",1973-06-30,200406,1886,427.617


In [5]:
pharma_df = pharma_df.sort_values('Market Cap ($Billion)', ascending=False).drop(columns=['SEC filings','GICS Sector','Headquarters Location','Date first added',
                                                                                          'CIK','Founded'])
pharma_df

,Symbol,Security,GICS Sub-Industry,Market Cap ($Billion)
263,JNJ,Johnson & Johnson,Pharmaceuticals,427.617
310,MRK,Merck & Co.,Pharmaceuticals,199.012
366,PFE,Pfizer Inc.,Pharmaceuticals,193.682
286,LLY,Lilly (Eli) & Co.,Pharmaceuticals,189.963
2,ABBV,AbbVie Inc.,Pharmaceuticals,183.185
504,ZTS,Zoetis,Pharmaceuticals,75.301
17,ALXN,Alexion Pharmaceuticals,Pharmaceuticals,34.079
472,VTRS,Viatris,Pharmaceuticals,21.518
88,CTLT,Catalent,Pharmaceuticals,19.680
365,PRGO,Perrigo,Pharmaceuticals,5.827


In [6]:
pharma_df['Price ($)']=''
pharma_df['Dividend ($)'] = ''
pharma_df['Yield (%)'] = ''
pharma_df['EPS (TTM)'] = ''
pharma_df['PE Ratio (TTM)'] = ''
pharma_df['PEG Ratio (5-Year)'] = ''
pharma_df['Price-Book Ratio'] = ''

pharma_df

,Symbol,Security,GICS Sub-Industry,Market Cap ($Billion),Price ($),Dividend ($),Yield (%),EPS (TTM),PE Ratio (TTM),PEG Ratio (5-Year),Price-Book Ratio
263,JNJ,Johnson & Johnson,Pharmaceuticals,427.617,,,,,,,
310,MRK,Merck & Co.,Pharmaceuticals,199.012,,,,,,,
366,PFE,Pfizer Inc.,Pharmaceuticals,193.682,,,,,,,
286,LLY,Lilly (Eli) & Co.,Pharmaceuticals,189.963,,,,,,,
2,ABBV,AbbVie Inc.,Pharmaceuticals,183.185,,,,,,,
504,ZTS,Zoetis,Pharmaceuticals,75.301,,,,,,,
17,ALXN,Alexion Pharmaceuticals,Pharmaceuticals,34.079,,,,,,,
472,VTRS,Viatris,Pharmaceuticals,21.518,,,,,,,
88,CTLT,Catalent,Pharmaceuticals,19.680,,,,,,,
365,PRGO,Perrigo,Pharmaceuticals,5.827,,,,,,,


In [7]:
for index, stock in pharma_df.iterrows():
    symbol = pharma_df.loc[index,'Symbol']
    pe = float(si.get_quote_table(symbol)['PE Ratio (TTM)'])
    eps = float(si.get_quote_table(symbol)['EPS (TTM)'])
    price = float(si.get_quote_table(symbol)['Quote Price'])
    pharma_df.loc[index,'PE Ratio (TTM)']= pe
    pharma_df.loc[index,'EPS (TTM)']= eps
    pharma_df.loc[index,'Price ($)']= price

In [8]:
for index, stock in pharma_df.iterrows():
    symbol = pharma_df.loc[index,'Symbol']
    peg = float(si.get_stats_valuation(symbol).iloc[4,1])
    pb = float(si.get_stats_valuation(symbol).iloc[6,1])
    pharma_df.loc[index,'PEG Ratio (5-Year)']= peg
    pharma_df.loc[index,'Price-Book Ratio']= pb

In [9]:
for index, stock in pharma_df.iterrows():
    symbol = pharma_df.loc[index,'Symbol']
    div = float(si.get_stats(symbol).set_index('Attribute').loc['Forward Annual Dividend Rate 4'])
    yld = float(si.get_stats(symbol).set_index('Attribute').loc['Forward Annual Dividend Yield 4'].str.replace('%',''))
    pharma_df.loc[index,'Dividend ($)']= div
    pharma_df.loc[index,'Yield (%)']= yld

In [10]:
pharma_df.style.background_gradient(cmap='viridis', low=0.5, high=0).highlight_null('red')

,Symbol,Security,GICS Sub-Industry,Market Cap ($Billion),Price ($),Dividend ($),Yield (%),EPS (TTM),PE Ratio (TTM),PEG Ratio (5-Year),Price-Book Ratio
263,JNJ,Johnson & Johnson,Pharmaceuticals,427.617000,162.395004,4.040000,2.480000,5.510000,29.480000,2.380000,6.640000
310,MRK,Merck & Co.,Pharmaceuticals,199.012000,78.695000,2.600000,3.360000,4.530000,17.380000,1.160000,6.680000
366,PFE,Pfizer Inc.,Pharmaceuticals,193.682000,34.869999,1.560000,4.360000,1.540000,22.670000,4.870000,3.060000
286,LLY,Lilly (Eli) & Co.,Pharmaceuticals,189.963000,198.619995,3.400000,1.670000,6.790000,29.260000,1.480000,41.210000
2,ABBV,AbbVie Inc.,Pharmaceuticals,183.185000,103.779999,5.200000,5.080000,4.570000,22.700000,nan,11.850000
504,ZTS,Zoetis,Pharmaceuticals,75.301000,158.354996,1.000000,0.640000,3.470000,45.640000,3.500000,20.350000
17,ALXN,Alexion Pharmaceuticals,Pharmaceuticals,34.079000,155.824997,nan,nan,4.310000,36.160000,0.970000,3.030000
472,VTRS,Viatris,Pharmaceuticals,21.518000,17.695000,nan,nan,nan,nan,nan,1.690000
88,CTLT,Catalent,Pharmaceuticals,19.680000,119.489998,nan,nan,1.580000,75.530000,3.250000,6.170000
365,PRGO,Perrigo,Pharmaceuticals,5.827000,42.650002,0.900000,2.060000,-0.050000,nan,nan,1.020000


In [11]:
pharma_df['EPS (TTM)'] = pharma_df['EPS (TTM)'].astype(float)
pharma_df['PE Ratio (TTM)'] = pharma_df['PE Ratio (TTM)'].astype(float)
pharma_df['PEG Ratio (5-Year)'] = pharma_df['PEG Ratio (5-Year)'].astype(float)
pharma_df['Price-Book Ratio'] = pharma_df['Price-Book Ratio'].astype(float)
pharma_df['Dividend ($)'] = pharma_df['Dividend ($)'].astype(float)
pharma_df['Yield (%)'] = pharma_df['Yield (%)'].astype(float)
pharma_df['Price ($)'] = pharma_df['Price ($)'].astype(float)
pharma_df.to_csv('Resources/pharma.csv', index=False)
pharma_df.dtypes

Symbol                    object
Security                  object
GICS Sub-Industry         object
Market Cap ($Billion)    float64
Price ($)                float64
Dividend ($)             float64
Yield (%)                float64
EPS (TTM)                float64
PE Ratio (TTM)           float64
PEG Ratio (5-Year)       float64
Price-Book Ratio         float64
dtype: object

In [12]:
#Create data frame with date index 
pharma_data = get_data('JNJ', start_date='01/01/2015', end_date='12/31/2020', interval ='1d').index
pharma_data_df = pd.DataFrame(index=pharma_data)
pharma_data_df

#Find daily close, volume and calculate returns for all symbols and then add to dataframe
for index, stock in pharma_df.iterrows():
    symbol = pharma_df.loc[index,'Symbol']#f"{stock['Symbol']}"
    symbol_data= get_data(symbol, start_date='01/01/2015', end_date='12/31/2020', interval ='1d')[['close','volume']]
    symbol_data=symbol_data.rename(columns={'close' : f"{symbol}_close", 'volume' : f"{symbol}_volume"})
    symbol_data[f"{symbol}_return"] = symbol_data[f"{symbol}_close"].diff() /symbol_data[f"{symbol}_close"]
    pharma_data_df = pd.merge(pharma_data_df, symbol_data, left_index=True, right_index=True)

pharma_data_df.to_csv('Resources/daily_pharma_data.csv')
pharma_data_df

,JNJ_close,JNJ_volume,JNJ_return,MRK_close,MRK_volume,MRK_return,PFE_close,PFE_volume,PFE_return,LLY_close,...,ALXN_return,VTRS_close,VTRS_volume,VTRS_return,CTLT_close,CTLT_volume,CTLT_return,PRGO_close,PRGO_volume,PRGO_return
2015-01-02,104.519997,5753600,NaN,57.189999,7076300,NaN,29.724857,16371571,NaN,70.059998,...,NaN,56.349998,1398000,NaN,27.510000,187300,NaN,166.630005,710700,NaN
2015-01-05,103.790001,8079300,-0.007033,58.040001,15961200,0.014645,29.563566,24786391,-0.005456,69.370003,...,-0.024318,55.720001,2996200,-0.011306,28.080000,455200,0.020299,164.910004,725500,-0.010430
2015-01-06,103.279999,7428000,-0.004938,60.320000,24287700,0.037798,29.810247,29468681,0.008275,69.720001,...,-0.023715,54.180000,6610900,-0.028424,26.549999,622100,-0.057627,164.600006,878500,-0.001883
2015-01-07,105.559998,7931700,0.021599,61.610001,18634500,0.020938,30.218216,20248816,0.013501,69.230003,...,0.053105,56.110001,3663000,0.034397,27.809999,457200,0.045307,167.350006,851300,0.016433
2015-01-08,106.389999,9916000,0.007802,62.849998,19121000,0.019729,30.834915,49169522,0.020000,70.870003,...,-0.022470,56.650002,3164000,0.009532,27.990000,183800,0.006431,171.169998,1117200,0.022317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,151.940002,4607300,-0.005134,79.760002,5778700,0.004263,37.439999,36182000,0.018697,165.479996,...,-0.003720,17.879999,6303400,0.007830,104.860001,618900,-0.000286,43.660000,904600,-0.004352
2020-12-24,152.470001,2114900,0.003476,80.139999,2822000,0.004742,37.270000,14790100,-0.004561,166.660004,...,-0.003997,17.770000,1946600,-0.006190,105.309998,277800,0.004273,44.009998,379100,0.007953
2020-12-28,153.190002,3855500,0.004700,80.449997,4584200,0.003853,36.820000,26993700,-0.012222,166.500000,...,0.011208,17.740000,4933900,-0.001691,102.879997,582000,-0.023620,44.650002,686700,0.014334
2020-12-29,154.139999,5212000,0.006163,81.419998,5446600,0.011914,37.049999,23152100,0.006208,166.580002,...,0.006373,17.750000,4811800,0.000563,102.709999,549200,-0.001655,44.549999,606700,-0.002245


In [29]:
#Find SPY data
SPY_data = get_data('SPY', start_date='01/01/2015', end_date='12/31/2020', interval ='1d')
SPY_data['SPY_return'] = SPY_data["close"].diff() /SPY_data["close"]
SPY_data = SPY_data.rename(columns={'close': 'SPY_close'})
SPY_data.to_csv('Resources/daily_SPY_data.csv')
SPY_data

,open,high,low,SPY_close,adjclose,volume,ticker,SPY_return
2015-01-02,206.380005,206.880005,204.179993,205.429993,182.746567,121465900,SPY,NaN
2015-01-05,204.169998,204.369995,201.350006,201.720001,179.446259,169632600,SPY,-0.018392
2015-01-06,202.089996,202.720001,198.860001,199.820007,177.756073,209151400,SPY,-0.009509
2015-01-07,201.419998,202.720001,200.880005,202.309998,179.971115,125346700,SPY,0.012308
2015-01-08,204.009995,206.160004,203.990005,205.899994,183.164703,147217800,SPY,0.017436
...,...,...,...,...,...,...,...,...
2020-12-23,368.279999,369.619995,367.220001,367.570007,367.570007,46201400,SPY,0.000898
2020-12-24,368.079987,369.029999,367.450012,369.000000,369.000000,26457900,SPY,0.003875
2020-12-28,371.739990,372.589996,371.070007,372.170013,372.170013,39000400,SPY,0.008518
2020-12-29,373.809998,374.000000,370.829987,371.459991,371.459991,53680500,SPY,-0.001911


In [28]:
#Create close price data frame for all pharma symbols and SPY
close_df = pharma_data_df[['JNJ_close',
                                     'PFE_close',
                                    'LLY_close',
                                    'PRGO_close', 
                                    'CTLT_close',                                  
                                    'VTRS_close',                                 
                                    'ALXN_close',                                  
                                    'ZTS_close',                                    
                                    'ABBV_close',                                  
                                    'MRK_close']]     
close_df=close_df.rename(columns={'SPY_close': 'SPY',
                                    'JNJ_close': 'JNJ',
                                    'PFE_close': 'PFE',
                                    'LLY_close': 'LLY',
                                    'PRGO_close': 'PRGO', 
                                    'CTLT_close': 'CTLT',                                  
                                    'VTRS_close': 'VTRS',                                 
                                    'ALXN_close': 'ALXN',                                  
                                    'ZTS_close': 'ZTS',                                    
                                    'ABBV_close': 'ABBV',                                  
                                    'MRK_close': 'MRK'})   
close_df.to_csv('Resources/close_price_data.csv')
close_df   

,JNJ,PFE,LLY,PRGO,CTLT,VTRS,ALXN,ZTS,ABBV,MRK
2015-01-02,104.519997,29.724857,70.059998,166.630005,27.510000,56.349998,186.600006,43.310001,65.889999,57.189999
2015-01-05,103.790001,29.563566,69.370003,164.910004,28.080000,55.720001,182.169998,43.049999,64.650002,58.040001
2015-01-06,103.279999,29.810247,69.720001,164.600006,26.549999,54.180000,177.949997,42.630001,64.330002,60.320000
2015-01-07,105.559998,30.218216,69.230003,167.350006,27.809999,56.110001,187.929993,43.509998,66.930000,61.610001
2015-01-08,106.389999,30.834915,70.870003,171.169998,27.990000,56.650002,183.800003,44.180000,67.629997,62.849998
...,...,...,...,...,...,...,...,...,...,...
2020-12-23,151.940002,37.439999,165.479996,43.660000,104.860001,17.879999,153.240005,159.850006,103.279999,79.760002
2020-12-24,152.470001,37.270000,166.660004,44.009998,105.309998,17.770000,152.630005,160.720001,103.260002,80.139999
2020-12-28,153.190002,36.820000,166.500000,44.650002,102.879997,17.740000,154.360001,162.389999,103.449997,80.449997
2020-12-29,154.139999,37.049999,166.580002,44.549999,102.709999,17.750000,155.350006,163.119995,104.699997,81.419998


In [33]:
#Create returns dataframe
returns_df = pd.merge(pharma_data_df[['JNJ_return',
                                     'PFE_return',
                                    'LLY_return',
                                    'PRGO_return', 
                                    'CTLT_return',                                  
                                    'VTRS_return',                                 
                                    'ALXN_return',                                  
                                    'ZTS_return',                                    
                                    'ABBV_return',                                  
                                    'MRK_return']],SPY_data['SPY_return'], left_index=True, right_index=True)                                                           
returns_df=returns_df.rename(columns={'SPY_return': 'SPY',
                                    'JNJ_return': 'JNJ',
                                    'PFE_return': 'PFE',
                                    'LLY_return': 'LLY',
                                    'PRGO_return': 'PRGO', 
                                    'CTLT_return': 'CTLT',                                  
                                    'VTRS_return': 'VTRS',                                 
                                    'ALXN_return': 'ALXN',                                  
                                    'ZTS_return': 'ZTS',                                    
                                    'ABBV_return': 'ABBV',                                  
                                    'MRK_return': 'MRK'})    
returns_df.to_csv('Resources/daily_returns.csv')
returns_df

,JNJ,PFE,LLY,PRGO,CTLT,VTRS,ALXN,ZTS,ABBV,MRK,SPY
2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,-0.007033,-0.005456,-0.009947,-0.010430,0.020299,-0.011306,-0.024318,-0.006040,-0.019180,0.014645,-0.018392
2015-01-06,-0.004938,0.008275,0.005020,-0.001883,-0.057627,-0.028424,-0.023715,-0.009852,-0.004974,0.037798,-0.009509
2015-01-07,0.021599,0.013501,-0.007078,0.016433,0.045307,0.034397,0.053105,0.020225,0.038847,0.020938,0.012308
2015-01-08,0.007802,0.020000,0.023141,0.022317,0.006431,0.009532,-0.022470,0.015165,0.010350,0.019729,0.017436
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,-0.005134,0.018697,-0.007493,-0.004352,-0.000286,0.007830,-0.003720,-0.012387,0.004648,0.004263,0.000898
2020-12-24,0.003476,-0.004561,0.007080,0.007953,0.004273,-0.006190,-0.003997,0.005413,-0.000194,0.004742,0.003875
2020-12-28,0.004700,-0.012222,-0.000961,0.014334,-0.023620,-0.001691,0.011208,0.010284,0.001837,0.003853,0.008518
2020-12-29,0.006163,0.006208,0.000480,-0.002245,-0.001655,0.000563,0.006373,0.004475,0.011939,0.011914,-0.001911


In [50]:
#Create 5 year average dividend yield data frame for JNJ, PFE, LLY and SPY
tickers = ['JNJ', 'PFE', 'LLY', 'SPY']
five_yr_avg_div = []

for ticker in tickers:
    symbol = yf.Ticker(ticker)
    five_yr_avg_div.append(symbol.info['fiveYearAvgDividendYield'])

five_yr_avg_div_dict = {'Symbol' : tickers,
                        '5yr avg Div' : five_yr_avg_div}
five_year_avg_div_df = pd.DataFrame(five_yr_avg_div_dict)
five_year_avg_div_df.to_csv('Resources/five_year_avg_div.csv')
five_year_avg_div_df

,Symbol,5yr avg Div
0,JNJ,2.63
1,PFE,3.65
2,LLY,2.28
3,SPY,NaN


In [54]:
#Create data frame yearly price data frame for JNJ, PFE, LLY and SPY
yearly_price_df = close_df[['JNJ','PFE','LLY']]
yearly_price_df['SPY'] = SPY_data['SPY_close']
yearly_price_df = yearly_price_df.loc[(yearly_price_df.index == '2016-01-04')| 
                    (yearly_price_df.index == '2017-01-03')|
                    (yearly_price_df.index == '2018-01-02')|
                    (yearly_price_df.index == '2019-01-02')|
                    (yearly_price_df.index == '2020-01-02')|
                    (yearly_price_df.index == '2020-12-30')
                   ]
yearly_price_df.to_csv('Resources/yearly_price.csv')
yearly_price_df

C:\Users\Noah\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,JNJ,PFE,LLY,SPY
2016-01-04,100.480003,30.313093,82.870003,201.020004
2017-01-03,115.839996,31.309298,74.599998,225.240005
2018-01-02,139.229996,34.573055,84.680000,268.769989
2019-01-02,127.750000,41.034157,114.879997,250.179993
2020-01-02,145.970001,37.134724,132.210007,324.869995
2020-12-30,156.050003,36.740002,167.009995,371.989990
